In [1]:
import networkx as nx
import random
import itertools

In [2]:
g = nx.read_edgelist("data/out.maayan-faa",create_using=nx.DiGraph(), nodetype = int)
# nx.info(g)
# count the number of nodes
print(g.number_of_nodes())
# number of self-nodes
print(g.selfloop_edges())

1226
<generator object selfloop_edges.<locals>.<genexpr> at 0x10acb91a8>


In [3]:
print(nx.info(g))

Name: 
Type: DiGraph
Number of nodes: 1226
Number of edges: 2615
Average in degree:   2.1330
Average out degree:   2.1330


In [45]:
#flajolet -martin
def trailing_zeroes(num):
    if num == 0:
        return 32 # Assumes 32 bit integer inputs!
    p = 0
    while (num >> p) & 1 == 0:
        p += 1
    return p

def estimate_cardinality(values, k):
    num_buckets = 2 ** k
    max_zeroes = [0] * num_buckets
    for value in values:
        h = hash(value)
        bucket = h & (num_buckets - 1) # Mask out the k least significant bits as bucket ID
        bucket_hash = h >> k
        max_zeroes[bucket] = max(max_zeroes[bucket], trailing_zeroes(bucket_hash))
    return 2 ** (float(sum(max_zeroes)) / num_buckets) * num_buckets * 0.79402

In [54]:
edges_flat = list(itertools.chain(*g.edges()))
estimate_cardinality(g.edges(), 10)

2574.9580071735936

In [36]:
len(edges_flat)

5230